# EEG - Flow

## X. Check all of the annotated signal, in search of wrong annotations labels

Step 1: 
Going through all files and look for wrong labels

Step 2: 
Selectively rename labels where needed

Last edit: 18.08.2024 15:50

# Step 1: Going through all files and look for wrong labels

In [1]:
from mne import set_log_level as set_log_level_mne
from mne.io import read_raw_fif

from eeg_flow import logger, set_log_level
from eeg_flow.config import load_config
from eeg_flow.utils.bids import get_derivative_folder, get_fname

set_log_level_mne("WARNING")
set_log_level("INFO")  # change to WARNING to print only bad files

Find all step-3 files in the dataset.

In [2]:
_, derivatives, _ = load_config()
files = derivatives.glob("**/*_step3_with-bads_raw.fif")

Load each file and check the annotation labels.

In [ ]:
bad_files = list()
for file in files:
    raw = read_raw_fif(file)

    # parse participant, task and run information
    key_info = file.split("\\")[4].split("_")
    participant = key_info[0][4:]
    task = key_info[1].split("-")[1]
    run = key_info[2].split("-")[1]

    # check annotation labels
    labels = set(raw.annotations.description)
    if len(set(labels).difference({"bad_", "standard", "novel", "target"})) != 0:
        logger.warning(
            "Participant %s, task %s, run %s has erroneous labels: %s",
            participant,
            task,
            run,
            labels,
        )
        bad_files.append(file)
    else:
        logger.info(
            "Participant %s, task %s, run %s: OK%s",
            participant,
            task,
            run,
            "" if "bad_" in labels else " (no bads)",
        )

Summary of files to amend:

In [ ]:
for file in bad_files:
    print(file)

# Step 2: Selectively rename labels where needed

Select the participant, task and run you want to update.

In [ ]:
participant_group: str = "P48-G8"
task: str = "UT"
run: int = 2

In [ ]:
participant = participant_group.split("-")[0]
group = participant_group.split("-")[1]
derivatives_folder = get_derivative_folder(derivatives, participant, group, task, run)
fname_stem = get_fname(participant, group, task, run)
fname = derivatives_folder / f"{fname_stem}_step3_with-bads_raw.fif"
raw = read_raw_fif(fname)
print (set(raw.annotations.description))

Use `annotation.rename{"XXX": "bad_"}` to rename the annotations.

In [ ]:
raw.annotations.rename({"badswronglabel_": "bad_"})
print (set(raw.annotations.description))

In [ ]:
fname_write = derivatives_folder / f"{fname_stem}_step3_with-bads_bis_raw.fif"
raw.save(fname_write, overwrite=False)